# Hyperparameter Tuning by Grid-Search

In the previous notebook, we saw that hyperparameters can affect the generalization performance of a model. In this notebook, we will show how to optimize hyperparameters using  grid-search approach

## Our Predictive Model

Let's reload the dataset as we did previously:

In [1]:
import pandas as pd

from sklearn import set_config
set_config(display="digram")

In [2]:
df = pd.read_csv('data/adult-census.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
target = df['class']
data = df.drop(columns=['class', 'education-num', 'fnlwgt'])

Once the dataset is loaded, we split it into a training and testing sets.

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
data_train, data_test, target_train, target_test = train_test_split(data, target,
                                                                   random_state=42)

We will define a pipeline as seen in the first module. It will handle both numerical and categorical features.  
The first step is to select all the categorical columns.

In [6]:
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include='object')
categorical_columns = categorical_columns_selector(data)

Here we will use a tree-based model as a classifier (i.e `HisGradientBoostingClassifier`). That means:

* Numerical variables don't need scaling;
* Categorical variables can be dealt with an `OrdinalEncoder` even if the coding order is not meaningful;
* For tree-based models, the `OrdinaleEncoder` avoids having high-dimensional representations.

We now build our `OrdinalEncoder` by passing it the known categories.

In [7]:
from sklearn.preprocessing import OrdinalEncoder

categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value", 
                                          unknown_value=-1)

We then use a `ColumnTransformer` to select the categorical columns and apply the `OrdinalEncoder` to them.

In [8]:
from sklearn.compose import ColumnTransformer

In [9]:
preprocessor = ColumnTransformer([
    ('cat_preprocessor', categorical_preprocessor, categorical_columns)],
    remainder='passthrough', sparse_threshold=0)

Finally, we use a tree-based classifier (i.e. histogram gradient-boosting) to predict whether or not a person earns more than 50k\\$ a year.

In [10]:
# for the moment this line is reuqired to import HistGradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

In [11]:
model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', HistGradientBoostingClassifier(random_state=42, max_leaf_nodes=4))
])

In [12]:
display(model)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                                   transformers=[('cat_preprocessor',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier',
                 HistGradientBoostingClassifier(max_leaf_nodes=4,
                                                random_state=42))])